# Lesson 3 - Implement Calibration (Spinup, Calibrate, Validate)

## Reminder From Lesson 2:
We covered the creation of the database and initialization of job/experiment. After initialization, each Domain specified in the `Domain_Meta` table is now populated as a subdirectory named by the `gageID` listed in table `Domain_Meta`, at the location specified in `setup.parm` file. Each Domain Directory contains the following subdirectories:

| SubDirectory Name | Description |
| ------------- | ------------- |
| FORCING | A symbolic link to the forcing directory for this particular basin |
| OBS | The directory containing symbolic links to the observation files necessary for the calibration workflow |
| RUN.CALIB | The directory that contains output for the calibration iterations. |
|RUN.SPINUP|The directory that contains output for the calibration spinup. |
|RUN.VALID|The directory that contains output for the calibration validation.|

In [ ]:
%%bash 
ls /home/docker/example_case/Calibration/output/example1/01447720

## Spinup
After successfully initializing model, the next step is to run a spinup period for each of the domains. This is handled by the script: `spinOrchestrator.py`. 
All the user spinup settings have been previously set in `setup.parm` file. 

|Argument|Required|Description|
|-|-|-|
|**coldStart** | Req. |Flag to direct calibration workflow to cold start your model simulation for each iteration during calibration. Specify 1 to activate. |
|**optSpinFlag** | Req. | Flag to direct the workflow to use an alternative spinup file already in place in the input directory for the basins being calibrated. This allows the user to bypass the spinup step. Specify 1 to activate. |
|**bSpinDate** | Opt. |Beginning date for the spinup. |
|**eSpinDate** | Opt. |Ending date for the spinup. |

**NOTE**: if optSpinFlag is set to 1, user must provide BOTH a hydro and land spinup state in the basin domain directories for ALL basins being used in this experiment. Expected file naming conventions for the expected restart files are as follows:
* Land:  LandRestartSubstitute.nc
* Hydro: HydroRestartSubstitute.nc

To initiate model spinup, run the following on the commandline (or in the cell provided):

In [ ]:
%%bash
python /home/docker/PyWrfHydroCalib/spinOrchestrator.py 1 --optDbPath /home/docker/example_case/Calibration/output/DATABASE.db

**Note**: ALL basins in a given job/experiment must complete their spinup for the program to complete successfully. The spinup program must complete before moving onto the parameter calibration. If you navigate to the `RUN.SPINUP` subdirectory under your basin, you will see an `OUTPUT` subdirectory. All files necessary to run the model for your spinup have been constructed here. This includes the namelist files, links to parameter files and executables, along with the spinup output. Let us check the content of the directory. 


In [ ]:
%%bash
# navigate to spinup directory here, and show the OUTPUT file location
ls /home/docker/example_case/Calibration/output/example1/01447720/RUN.SPINUP/OUTPUT


If for any reason (like not having the right WRF-Hydro executable, or an incompatable namelist options) the spin up model runs fails, the workflow will create a lock file called `RUN.LOCK` file which will be on the `OUTPUT` directory. If you have specified the `email` argument in the setup file, then the workflow will send out an email to user and specify where the lock file is located (not set up in this training image). User needs to checkout the run directory (`OUTPUT`), identify why the model run failed, address the issue and then remove the `RUN.LOCK` file. After removing the lock file, the python workflow (`spinOrchestrator.py`) will restart from it was left and restart the model run. When spin up for all the basins in a given job/experiment is finished it will send an email to user with the following content: 

*SPINUP FOR JOB ID: 1 COMPLETE.*

Upon receiving this email, user could move to next step and start the calibration process. Note, if you submit the calibration before spin up is finished, it will throw an error that the spin up has not yet finished and you cannot submit the calibration. 

## Calibration:
You are now ready to calibrate your model parameters for your basins. To do this, we will run `calibOrchestrator.py`. 

### What Happens:
The workflow will proceed to manage running each calibration iteration, which includes launching model jobs, adjusting parameter files used by the model, running analysis code to calculate new parameter values, and generating updated plots. The workflow will be monitoring each stage of the iteration to ensure things complete successfully. In order for the calibration to complete successfully, the workflow must complete all iterations for every basin in your experiment. 

The process of launching the calibration step is the same as the spinup step. 
Navigate to your calibration python code directory and run the following:

In [ ]:
%%bash
python /home/docker/PyWrfHydroCalib/calibOrchestrator.py 1 --optDbPath /home/docker/example_case/Calibration/output/DATABASE.db

*Note: JobID is the unique job ID value created when you initialized your experiment. The calibration program must complete before you can run the validation program.*

In [ ]:
%%bash
# Now lets view output files resulting from the Calibration step:

# navigate to spinup directory here, and show the OUTPUT file location
ls /home/docker/example_case/Calibration/output/example1/01447720/RUN.CALIB/OUTPUT

#### Changes made by calibOrchestratory.py to RUN.CALIB directory:

|Run.Calib Subdirectories|Description|
|-|-|
|calib_parm.tbl|copy of calibration parameter table specified during jobInit.py|
|Multiple*|symbolic links to executables for R/Python/executables|
|calibScript.R|a temporary R namelist file that is updated by the workflow in-between model iterations|
|proj_data.Rdata|File created by R code to hold model statistics and parameter values as they are updated throughout the calibration process.|
|BASELINE_PARAMETERS|directory contains the original parameter files specified during the domain input process. These parameter files include the groundwater parameter file, the 2D Hydro file, along with the Fulldom and soil properties file|
|DEFAULT_PARAMETERS|The same as BASELINE, but with the default values applied to them from the calibration parameters table|
|FINAL_PARAMETERS|directory contains the same adjusted files, but with the final calibrated parameter values applied to them.|
|plots|This directory contains several PNG plots that can be very useful. A more detailed explanation of some of the products in this sub-directory are explained in lesson 4 to follow.|
|OUTPUT|sub-directory containing model output for the last iteration. These files are very dynamic and get overwritten during each model iteration that is run|


#### Changes to Database made by calibOrchestratory.py:

|Database Table|Description of Change|
|-|-|
|Calib_Params | Tables are updated dynamically by the workflow as the calibration progresses. The ‘Calib_Params’ table will contain the parameter values for each model iteration.|
|Calib_Stats | Table contains error metrics for each model iteration as parameter values are adjusted. Many of these values are visualized through the plots that are generated during the calibration process. However, having these values in the database allows you to perform further analysis with the data after the fact to better understand how the system evolved over time.|


When calibration for all the basins in a given job/experiment is finished it will send an email to user with the following content:

CALIBRATION FOR JOB ID: 1 COMPLETE.

## Validation
You are now ready to validate your calibrated parameter values for your basins via `validOrchestrator.py` . The process of running the validation is exactly the same as with the spinup and calibration.

In your terminal session, or in the cell below, execute the following command:

In [ ]:
%%bash
python /home/docker/PyWrfHydroCalib/runValidOrchestrator.py  /home/docker/PyWrfHydroCalib/ 1 --optDbPath /home/docker/example_case/Calibration/output/DATABASE.db

*Note: JobID is the unique job ID value created when you initialized your experiment.*

In [ ]:
%%bash
# Now lets view output files resulting from the Valibration step:
ls /home/docker/example_case/Calibration/output/example1/01447720/RUN.VALID/OUTPUT/

Within the RUN.VALID/OUTPUT directory, there are two subdirectories that contain the restart and model run information for the default (CTRL) and Best (BEST) performing simulation runs. Contents of each shown below: 

* RUN.VALID/OUTPUT/BEST --> contains the restart and model run information for the best performing parameterization

* RUN.VALID/OUTPUT/CTRL --> contains the restart and model run with default paramter values

In [ ]:
%%bash
# Now lets view output files resulting from the Valibration step:
ls /home/docker/example_case/Calibration/output/example1/01447720/RUN.VALID/OUTPUT/*/

### What happens: 

The workflow will then proceed to manage running NWM/WRF-Hydro over your basins with the default parameter values specified at the beginning of the experiment, along with your calibrated parameter values. 

Under the `RUN.VALID` subdirectory for each basin, you will see several files and sub-directories as you did with the `RUN.CALIB` directory. 

## Conclusion:
Now all the steps are complete, we will take a look at the results and plots next.